**Lip synchronization dubbing from albanian to [language]**

Imports

In [ ]:
import os
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
from google.api_core.client_options import ClientOptions
from google.cloud import speech_v2
from google.api_core.exceptions import NotFound

Google cloud data

In [ ]:
PROJECT_ID = "890676014334"
LOCATION = "europe-west4"

# Chirp 2 is only available in certain locations
client_options_var = ClientOptions(api_endpoint="europe-west4-speech.googleapis.com")

# Initialize the client
client = speech_v2.SpeechClient(client_options=client_options_var)
recogniser = f"projects/{PROJECT_ID}/locations/{LOCATION}/recognizers/albanian-recogniser"

Transcribing the videos

In [ ]:
video_file = "path_to_video" # CHANGE VIDEO FILE PATH

audio_path = "temp_audio.wav"
video_file.audio.write_audiofile(audio_path, fps=16000, logger=None)
 
# SAMPLE ROW
# file_name:confidence_score:transcription
# video_file_1.mp4:0.72:Po perse more djale e bere kete gje jo te ndershme.

# Base name for the transcription file
base_name = "transcription_alb.txt"

try:
    config = speech_v2.RecognitionConfig(
        auto_decoding_config=speech_v2.AutoDetectDecodingConfig(),
        language_codes=["sq-AL"], 
        model="chirp_2",
        features=cloud_speech.RecognitionFeatures(
            enable_word_time_offsets=True, # this is not tested and good chance it does not work
        ),
    )
            
    request = speech_v2.RecognizeRequest(
        recognizer=recogniser,
        config=config,
        content=audio_path
    )
            
    # Transcribes the audio into text
    response = client.recognize(request=request)

    # with a mode, append, we only add data at the end of the pointer.
    with open(base_name, "a", encoding="utf-8") as f:
        f.write(f"{video_file}:{alternatives.confidence}:{alternatives.transcript}\n")

except Exception as e:
    print(f"Error: {e}")    
    print("\nRecognizer not found, bad name or something...")

# Remove the temp audio file after extracting it from the video
os.remove(audio_path)


Translating the transcription file

In [ ]:
from googletrans import Translator

input_file = "transcription_alb.txt"
output_file = "transcription_eng.txt"

target_language = "en" # en (english), de (deutch), ja (japonese)
translator = Translator()

try:
    with open(input_file, 'r', encoding='utf-8') as infile, \
        open(output_file, 'w', encoding='utf-8') as outfile:
        for line in infile:
            line = line.strip()
            if line:  # Ensure the line is not empty
                parts = line.split(':', 2)  # Split into at most 3 parts
                if len(parts) == 3:
                    file_name, confidence_score, transcription_text = parts
                    try:
                        translation = translator.translate(transcription_text, src='sq', dest=target_language)
                        translated_line = f"{file_name}:{confidence_score}:{translation.text}"
                        outfile.write(translated_line + '\n')
                    except Exception as e:
                        print(f"Error translating line: {line} - {e}")
                        outfile.write(line + '\n')  # Write the original line if translation fails
                else:
                    print(f"Skipping invalid line format: {line}")
                    outfile.write(line + '\n')  # Write the original line if format is wrong
except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Voice cloning from the transcription\own voice